In [1]:
import torch
import numpy as np
import pandas as pd
import random
import copy 
import time 

In [ ]:
# 先写麻烦的 那就是要确保流量不变性
# 确定一下流量的比例 到底这个初始的流量比例占百分之多少 然后输出一下
# 把mac_sev s_cpu s_mem totoal_cpu total_mem 读进来 其实最简单的办法就是把mac_sev的每一种服务增加一下 然后直到物理机容量增加

In [2]:
ll_path = "../../"
rank_name = ll_path + 'rank.txt'

rank_array = np.loadtxt(rank_name, dtype=np.float32, delimiter=',')
rank = int(rank_array[2])

reward_kind_name = 'reward.txt'
with open(reward_kind_name, 'r') as f:
    reward_kind = f.read()

filename = ll_path + 'data' + str(rank) + '/'
dot = '.npy'
mac_kind = int(rank_array[0])
sev_kind = int(rank_array[1])


mac_sev_name = 'mac_sev' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
mac_trf_name = 'mac_trf' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
sev_trf_name = 'sev_trf' + '_' + str(sev_kind) + '_' + str(rank)  # 后面可以补充字符串说明
sev_trf_out_name = 'sev_trf_out' + '_' + \
    str(sev_kind) + '_' + str(rank)  # 后面可以补充字符串说明
sev_trf_in_name = 'sev_trf_in' + '_' + \
    str(sev_kind) + '_' + str(rank)  # 后面可以补充字符串说明
sev_cpu_name = 'sev_cpu' + '_' + str(sev_kind) + '_' + str(rank)
sev_mem_name = 'sev_mem' + '_' + str(sev_kind) + '_' + str(rank)
mac_cpu_name = 'mac_cpu' + '_' + str(mac_kind) + '_' + str(rank)
mac_mem_name = 'mac_mem' + '_' + str(mac_kind) + '_' + str(rank)
total_cpu_name = 'total_cpu' + '_' + str(mac_kind) + '_' + str(rank)
total_mem_name = 'total_mem' + '_' + str(mac_kind) + '_' + str(rank)
mac_sev_cpu_name = 'mac_sev_cpu' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
mac_sev_mem_name = 'mac_sev_mem' + '_' + \
    str(mac_kind) + '_' + str(sev_kind) + '_' + str(rank)
now_all_name = 'now_all' + '_' + str(mac_kind) + '_' + str(rank)

mac_cpu = np.load(filename + mac_cpu_name + dot)
mac_mem = np.load(filename + mac_mem_name + dot)
trf_cpu_mem = np.load(filename + now_all_name + dot)
mac_sev_cpu = np.load(filename + mac_sev_cpu_name + dot)
mac_sev_mem = np.load(filename + mac_sev_mem_name + dot)
mac_sev = np.load(filename + mac_sev_name + dot)
sev_trf = np.load(filename + sev_trf_name + dot)
sev_trf_out = np.load(filename + sev_trf_out_name + dot)
sev_trf_in = np.load(filename + sev_trf_in_name + dot)
total_cpu = np.load(filename + total_cpu_name + dot)
total_mem = np.load(filename + total_mem_name + dot)

s_cpu = np.load(filename + sev_cpu_name + dot)
s_mem = np.load(filename + sev_mem_name + dot)
stop = np.zeros(mac_sev.shape[0]).reshape((mac_sev.shape[0], 1))
trf_cpu_mem = np.concatenate((trf_cpu_mem, stop), axis=1)

topk = 0  # np.loadtxt('line2.csv', delimiter=",")


print(trf_cpu_mem.shape)
print(mac_sev_cpu.shape)
print(mac_sev_mem.shape)
print(mac_sev.shape)
print(sev_trf.shape)
print(sev_trf_out.shape)
print(sev_trf_in.shape)
print(total_cpu.shape)
print(total_mem.shape)
print(mac_cpu.shape)
print(mac_mem.shape)

(10, 8)
(10, 80)
(10, 80)
(10, 80)
(80, 80)
(80, 80)
(80, 80)
(10,)
(10,)
(10,)
(10,)


In [4]:
# 流量比
trf_index = np.where(sev_trf != 0)[0]

In [71]:
trf_index.shape

(2581,)

In [63]:
np.where(a > 0)

(array([0, 0, 1, 1], dtype=int64), array([0, 2, 0, 2], dtype=int64))

In [5]:
print("{} -- {} 一共有多少个流量对 {}".format(mac_kind, sev_kind, trf_index.shape[0]))
print("{} -- {} 流量对比例 {}".format(mac_kind, sev_kind, trf_index.shape[0] / (sev_trf.shape[0] * sev_trf.shape[0]) ))
print("{} -- {} 流量对比例 {}".format(mac_kind, sev_kind, trf_index.shape[0] / (sev_trf.shape[0] * (sev_trf.shape[0] - 1) / 2) ))

10 -- 80 一共有多少个流量对 2581
10 -- 80 流量对比例 0.40328125
10 -- 80 流量对比例 0.8167721518987342


In [6]:
print(np.mean(trf_cpu_mem[:, 3]))
print(np.mean(trf_cpu_mem[:, -2]))

0.5230830459211718
0.5110695278504783


## 分布变化 CPU、MEM、TRF均不变

In [7]:
def gen_mac_sev(t_mac_kind, t_sev_kind, t_sev_num_low, t_sev_num_high, rp1, rp2):
    zero = 0
    t_mac_sev = np.zeros((t_mac_kind, t_sev_kind))
    for i in range(t_mac_kind):
        for j in range(t_sev_kind):
            if random.random() < rp1:
                t_mac_sev[i][j] = 0
            else:
                t_mac_sev[i][j] = random.randint(t_sev_num_low, t_sev_num_high)
    
    for i in range(t_mac_kind):
        for j in range(t_sev_kind):
            if random.random() < rp2:
                t_mac_sev[i][j] = 0

    per_kinds = list(t_mac_sev.sum(axis = 0))
    for i in range(len(per_kinds)):
        if zero == per_kinds[i]:
            i_random = random.randint(zero, t_mac_sev.shape[0] - 1)
            t_mac_sev[i_random][i] = random.randint(max(t_sev_num_low, 1), t_sev_num_high)
    return t_mac_sev

def com_cpu_mem(t_total_cpu, t_total_mem, t_s_cpu, t_s_mem, t_mac_sev, t_mac_kind):
    mac_cpu = np.zeros(t_mac_kind)
    mac_mem = np.zeros(t_mac_kind)
    for i in range(t_mac_kind):
        for j in range(t_mac_sev.shape[1]):
            mac_cpu[i] += t_s_cpu[j] * t_mac_sev[i][j]
            mac_mem[i] += t_s_mem[j] * t_mac_sev[i][j]
    print(mac_cpu / t_total_cpu)
    print(mac_mem / t_total_mem)
    print(mac_cpu)
    print(mac_mem)
    a = np.all(mac_cpu < t_total_cpu)
    b = np.all(mac_mem < t_total_mem)
    print("mac < total? {} {}".format(a, b))
    return a, b, mac_cpu, mac_mem      

def compute_mac_trf(t_mac_kind, t_sev_kind, t_mac_sev, t_sev_trf_out, t_sev_trf_in):
    t_mac_trf = np.zeros(t_mac_kind)
    for i in range(t_mac_kind):
        for j in range(t_sev_kind - 1):
            for k in range(j + 1, t_sev_kind):
                t_mac_trf[i] += min(t_mac_sev[i][j] * t_sev_trf_out[j][k], t_mac_sev[i][k] * t_sev_trf_in[j][k])
    return t_mac_trf

def gtotal_and_now_cpumem(test_cpu, t_total_cpu, test_mem, t_total_mem, t_mac_trf):
    return np.stack((t_mac_trf, test_cpu, t_total_cpu, test_cpu/t_total_cpu, test_mem, t_total_mem, test_mem/t_total_mem), axis = 1)

def gen_ms_cpu_mem(t_mac_kind, t_sev_kind, t_mac_sev, t_sev_cpu, t_sev_mem):
    mac_sev_cpu = np.zeros((t_mac_kind, t_sev_kind))
    mac_sev_mem = np.zeros((t_mac_kind, t_sev_kind))
    for i in range(t_mac_kind):
        for j in range(t_sev_kind):
            if t_mac_sev[i][j] > 0:
                mac_sev_cpu[i][j] = t_sev_cpu[j]
                mac_sev_mem[i][j] = t_sev_mem[j]
    return mac_sev_cpu, mac_sev_mem

import math 
def lcm(x, y):
    a = int(x)
    b = int(y)
    return a * b // math.gcd(a, b)

def gen_sev_trf(t_sev_kind, t_trf_low, t_trf_high):
    t_sev_trf = np.zeros((t_sev_kind, t_sev_kind))
    for i in range(t_sev_kind - 1):
        for j in range(i + 1, t_sev_kind):
            if random.random() < 0.4:
                t_sev_trf[i][j] = 0
            else:
                t_sev_trf[i][j] = int(random.randint(t_trf_low, t_trf_high))
            #sev_trf[j][i] = sev_trf[i][j]
    return t_sev_trf

def gen_sev_trf_out_and_in(t_sev_trf, t_mac_sev, t_sev_kind, func):
    sev_trf_out = np.zeros((t_sev_kind, t_sev_kind))
    sev_trf_in = np.zeros((t_sev_kind, t_sev_kind))
    per_kinds_demand = t_mac_sev.sum(axis = 0)
    for i in range(t_sev_kind - 1):
        for j in range(i + 1, t_sev_kind):
            d_i = copy.deepcopy(per_kinds_demand[i])
            #print('Testdi')
            #print((i, d_i))
            d_j = copy.deepcopy(per_kinds_demand[j])
            #print('Testdj')
            #print((j, d_j))
            # 28, 3, 5 -- > 28 / 15 = 1 15, 3-5, 5-3
            #i --> j
            d = func(d_i, d_j)
            #print('d, sev_yrfij')
            #print(d)
            #print(sev_trf[i][j])
            t_sev_trf[i][j] = int(t_sev_trf[i][j] / d) * d
            sev_trf_out[i][j] = t_sev_trf[i][j] / d_i
            sev_trf_in[i][j] = t_sev_trf[i][j] / d_j
    return t_sev_trf, sev_trf_out, sev_trf_in


In [ ]:
def test():
    return 

In [8]:
path = "../../data1/"

## 1

In [9]:
re_mac_sev1 = gen_mac_sev(mac_kind, sev_kind, 0, 2, 0.4, 0.4)
sev_trf_old_1 = gen_sev_trf(sev_kind, 100, 1000)
sev_trf_new_1, sev_trf_out_1, sev_trf_in_1 = gen_sev_trf_out_and_in(sev_trf_old_1, re_mac_sev1, sev_kind, lcm)
m, n, mac_cpu1, mac_mem1 = com_cpu_mem(total_cpu, total_mem, s_cpu, s_mem, re_mac_sev1, mac_kind)
re_mac_trf_1 = compute_mac_trf(mac_kind, sev_kind, re_mac_sev1, sev_trf_out_1, sev_trf_in_1)
total_tcm_1 = gtotal_and_now_cpumem(mac_cpu1, total_cpu, mac_mem1, total_mem, re_mac_trf_1)
m_s_c1, m_s_m1 = gen_ms_cpu_mem(mac_kind, sev_kind, re_mac_sev1, s_cpu, s_mem)
print(np.mean(total_tcm_1[:,3]))
print(np.mean(total_tcm_1[:,-1]))

[0.35030232 0.43592744 0.29354398 0.3370392  0.23717574 0.26089981
 0.36595547 0.40584416 0.33279221 0.32444341]
[0.36803049 0.36141195 0.31407942 0.27196149 0.229643   0.26513872
 0.4159228  0.33486128 0.28057901 0.28733414]
[1796. 2235. 1505. 1728. 1216. 1125. 1578. 1750. 1435. 1399.]
[1835. 1802. 1566. 1356. 1145. 1099. 1724. 1388. 1163. 1191.]
mac < total? True True
0.334392375227659
0.31289623007146183


In [10]:
np.save(path + 'VT_1mac_sev_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', re_mac_sev1)
np.save(path + 'VT_1mac_trf_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', re_mac_trf_1)
np.save(path + 'VT_1now_all_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', total_tcm_1)
np.save(path + 'VT_1mac_sev_cpu' + str(mac_kind) + '_' + str(sev_kind) + '.npy', m_s_c1)
np.save(path + 'VT_1mac_sev_mem' + str(mac_kind) + '_' + str(sev_kind) + '.npy', m_s_m1)
np.save(path + 'VT_1sev_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_new_1)
np.save(path + 'VT_1sev_trf_out' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_out_1)
np.save(path + 'VT_1sev_trf_in' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_in_1)

## 2

In [11]:
re_mac_sev2 = gen_mac_sev(mac_kind, sev_kind, 0, 3, 0.4, 0.24)
sev_trf_old_2 = gen_sev_trf(sev_kind, 100, 1000)
sev_trf_new_2, sev_trf_out_2, sev_trf_in_2 = gen_sev_trf_out_and_in(sev_trf_old_2, re_mac_sev2, sev_kind, lcm)
m, n, mac_cpu2, mac_mem2 = com_cpu_mem(total_cpu, total_mem, s_cpu, s_mem, re_mac_sev2, mac_kind)
re_mac_trf_2 = compute_mac_trf(mac_kind, sev_kind, re_mac_sev2, sev_trf_out_2, sev_trf_in_2)
total_tcm_2 = gtotal_and_now_cpumem(mac_cpu2, total_cpu, mac_mem2, total_mem, re_mac_trf_2)
m_s_c2, m_s_m2 = gen_ms_cpu_mem(mac_kind, sev_kind, re_mac_sev2, s_cpu, s_mem)
print(np.mean(total_tcm_2[:,3]))
print(np.mean(total_tcm_2[:,-1]))

[0.51921201 0.53325531 0.46069826 0.4519212  0.38287498 0.60088126
 0.63520408 0.89053803 0.61363636 0.54684601]
[0.53650221 0.4947854  0.64981949 0.40513438 0.5        0.73510253
 0.72086852 0.81978287 0.7039807  0.62581423]
[2662. 2734. 2362. 2317. 1963. 2591. 2739. 3840. 2646. 2358.]
[2675. 2467. 3240. 2020. 2493. 3047. 2988. 3398. 2918. 2594.]
mac < total? True True
0.5635067522407654
0.6191790330174187


In [12]:
np.save(path + 'VT_2mac_sev_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', re_mac_sev2)
np.save(path + 'VT_2mac_trf_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', re_mac_trf_2)
np.save(path + 'VT_2now_all_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', total_tcm_2)
np.save(path + 'VT_2mac_sev_cpu' + str(mac_kind) + '_' + str(sev_kind) + '.npy', m_s_c2)
np.save(path + 'VT_2mac_sev_mem' + str(mac_kind) + '_' + str(sev_kind) + '.npy', m_s_m2)
np.save(path + 'VT_2sev_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_new_2)
np.save(path + 'VT_2sev_trf_out' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_out_2)
np.save(path + 'VT_2sev_trf_in' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_in_2)

## 3

In [14]:
re_mac_sev3 = gen_mac_sev(mac_kind, sev_kind, 0, 3, 0.3, 0.17)
sev_trf_old_3 = gen_sev_trf(sev_kind, 100, 1000)
sev_trf_new_3, sev_trf_out_3, sev_trf_in_3 = gen_sev_trf_out_and_in(sev_trf_old_3, re_mac_sev3, sev_kind, lcm)
m, n, mac_cpu3, mac_mem3 = com_cpu_mem(total_cpu, total_mem, s_cpu, s_mem, re_mac_sev3, mac_kind)
re_mac_trf_3 = compute_mac_trf(mac_kind, sev_kind, re_mac_sev3, sev_trf_out_3, sev_trf_in_3)
total_tcm_3 = gtotal_and_now_cpumem(mac_cpu3, total_cpu, mac_mem3, total_mem, re_mac_trf_3)
m_s_c3, m_s_m3 = gen_ms_cpu_mem(mac_kind, sev_kind, re_mac_sev3, s_cpu, s_mem)
print(np.mean(total_tcm_3[:,3]))
print(np.mean(total_tcm_3[:,-1]))

[0.79578701 0.67524868 0.6370197  0.70918666 0.75697289 0.69828386
 0.97820037 0.65909091 0.82954545 0.85064935]
[0.68351384 0.74047333 0.7452868  0.70537505 0.69173686 0.77539204
 0.77466828 0.69384801 0.74089264 0.6960193 ]
[4080. 3462. 3266. 3636. 3881. 3011. 4218. 2842. 3577. 3668.]
[3408. 3692. 3716. 3517. 3449. 3214. 3211. 2876. 3071. 2885.]
mac < total? True True
0.7589984884852393
0.724720614584528


In [15]:
np.save(path + 'VT_3mac_sev_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', re_mac_sev3)
np.save(path + 'VT_3mac_trf_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', re_mac_trf_3)
np.save(path + 'VT_3now_all_' + str(mac_kind) + '_' + str(sev_kind) + '.npy', total_tcm_3)
np.save(path + 'VT_3mac_sev_cpu' + str(mac_kind) + '_' + str(sev_kind) + '.npy', m_s_c3)
np.save(path + 'VT_3mac_sev_mem' + str(mac_kind) + '_' + str(sev_kind) + '.npy', m_s_m3)
np.save(path + 'VT_3sev_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_new_3)
np.save(path + 'VT_3sev_trf_out' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_out_3)
np.save(path + 'VT_3sev_trf_in' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_in_3)

## 流量变化 分布不变

In [73]:
# 之后要弄流量变化 这个太好说了 直接拿个新的流量矩阵就可以
# 直接在之前的流量利用率上进行改动就可以了
import math 
def lcm(x, y):
    a = int(x)
    b = int(y)
    return a * b // math.gcd(a, b)

def gen_sev_trf(t_sev_kind, t_trf_low, t_trf_high, tt_sev_trf, p, pr):
    # t_sev_trf = np.zeros((t_sev_kind, t_sev_kind))
    t_sev_trf = copy.deepcopy(tt_sev_trf)
    for i in range(t_sev_kind - 1):
        for j in range(i + 1, t_sev_kind):
            if t_sev_trf[i][j] != 0:
                if random.random() < pr:
                    t_sev_trf[i][j] = 0
            if t_sev_trf[i][j] == 0:
                if random.random() < p:
                    t_sev_trf[i][j] = int(random.randint(t_trf_low, t_trf_high))
            #sev_trf[j][i] = sev_trf[i][j]
    index = np.where(t_sev_trf != 0)[0]
    print( "流量利用率为{}".format(index.shape[0] / ( (t_sev_trf.shape[0] - 1) * t_sev_trf.shape[0] / 2) )) 
    print( "流量利用率为{}".format(index.shape[0] / (t_sev_trf.shape[0] ** 2)) )
    return t_sev_trf

def gen_sev_trf_out_and_in(t_sev_trf, t_mac_sev, t_sev_kind, func):
    sev_trf_out = np.zeros((t_sev_kind, t_sev_kind))
    sev_trf_in = np.zeros((t_sev_kind, t_sev_kind))
    per_kinds_demand = t_mac_sev.sum(axis = 0)
    for i in range(t_sev_kind - 1):
        for j in range(i + 1, t_sev_kind):
            d_i = copy.deepcopy(per_kinds_demand[i])
            #print('Testdi')
            #print((i, d_i))
            d_j = copy.deepcopy(per_kinds_demand[j])
            #print('Testdj')
            #print((j, d_j))
            # 28, 3, 5 -- > 28 / 15 = 1 15, 3-5, 5-3
            #i --> j
            d = func(d_i, d_j)
            #print('d, sev_yrfij')
            #print(d)
            #print(sev_trf[i][j])
            t_sev_trf[i][j] = int(t_sev_trf[i][j] / d) * d
            sev_trf_out[i][j] = t_sev_trf[i][j] / d_i
            sev_trf_in[i][j] = t_sev_trf[i][j] / d_j
    return t_sev_trf, sev_trf_out, sev_trf_in

## 1

In [77]:
s1 = gen_sev_trf(sev_kind, 100, 1000, sev_trf, 0, 0.25)
sev_trf_1, sev_trf_out_1, sev_trf_in_1 = gen_sev_trf_out_and_in(s1, mac_sev, sev_kind, lcm)
mt1 = compute_mac_trf(mac_kind, sev_kind, mac_sev, sev_trf_out_1, sev_trf_in_1)
now_all1 = gtotal_and_now_cpumem(mac_cpu, total_cpu, mac_mem, total_mem, mt1)
np.save(path + 'CT1_sev_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_1)
np.save(path + 'CT1_sev_trf_out' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_out_1)
np.save(path + 'CT1_sev_trf_in' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_in_1)
np.save(path + 'CT1_mac_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', mt1)
np.save(path + 'CT1_now_all' + str(mac_kind) + '_' + str(sev_kind) + '.npy', now_all1)

流量利用率为0.615506329113924
流量利用率为0.30390625


## 2

In [79]:
s2 = gen_sev_trf(sev_kind, 100, 1000, sev_trf, 0, 0.5)
sev_trf_2, sev_trf_out_2, sev_trf_in_2 = gen_sev_trf_out_and_in(s2, mac_sev, sev_kind, lcm)
mt2 = compute_mac_trf(mac_kind, sev_kind, mac_sev, sev_trf_out_2, sev_trf_in_2)
now_all2 = gtotal_and_now_cpumem(mac_cpu, total_cpu, mac_mem, total_mem, mt2)
np.save(path + 'CT2_sev_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_2)
np.save(path + 'CT2_sev_trf_out' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_out_2)
np.save(path + 'CT2_sev_trf_in' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_in_2)
np.save(path + 'CT2_mac_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', mt2)
np.save(path + 'CT2_now_all' + str(mac_kind) + '_' + str(sev_kind) + '.npy', now_all2)

流量利用率为0.41107594936708863
流量利用率为0.20296875


## 3

In [82]:
s3 = gen_sev_trf(sev_kind, 100, 1000, sev_trf, 0, 0.75)
sev_trf_3, sev_trf_out_3, sev_trf_in_3 = gen_sev_trf_out_and_in(s3, mac_sev, sev_kind, lcm)
mt3 = compute_mac_trf(mac_kind, sev_kind, mac_sev, sev_trf_out_3, sev_trf_in_3)
now_all3 = gtotal_and_now_cpumem(mac_cpu, total_cpu, mac_mem, total_mem, mt3)
np.save(path + 'CT3_sev_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_3)
np.save(path + 'CT3_sev_trf_out' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_out_3)
np.save(path + 'CT3_sev_trf_in' + str(mac_kind) + '_' + str(sev_kind) + '.npy', sev_trf_in_3)
np.save(path + 'CT3_mac_trf' + str(mac_kind) + '_' + str(sev_kind) + '.npy', mt3)
np.save(path + 'CT3_now_all' + str(mac_kind) + '_' + str(sev_kind) + '.npy', now_all3)

流量利用率为0.20537974683544305
流量利用率为0.10140625
